In [7]:
import cv2
import numpy as np
import mediapipe as mp
import math
from screen_brightness_control import set_brightness
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import comtypes


# Setup MediaPipe hands
mpHands = mp.solutions.hands
hands = mpHands.Hands()
draw = mp.solutions.drawing_utils

In [ ]:
def get_landmark(frame, processed, draw, hands):
    
    left_landmark = []   # Stores left hand landmarks (index, x, y)
    right_landmark = []  # Stores right hand landmarks (index, x, y)

    # Check if hands are detected
    if processed.multi_hand_landmarks:
        for hand_index, handlm in enumerate(processed.multi_hand_landmarks):
            # Get label from MediaPipe: "Left" or "Right"
            hand_label = processed.multi_handedness[hand_index].classification[0].label
            actual_label = "Right" if hand_label == "Left" else "Left"

            # Loop over all 21 hand landmarks
            for index, landmark in enumerate(handlm.landmark):
                h, w, _ = frame.shape  # Get frame dimensions

                #normalized cord to pixel values
                x = int(landmark.x * w)
                y = int(landmark.y * h)

                # Only store landmark 4,8
                if index in [4, 8]:
                    if actual_label == "Left":
                        left_landmark.append([index, x, y])
                    elif actual_label == "Right":
                        right_landmark.append([index, x, y])

            draw.draw_landmarks(frame, handlm, mpHands.HAND_CONNECTIONS)

    return left_landmark, right_landmark


In [ ]:
def get_distance(frame, landmark_list):

    if len(landmark_list) != 2:
        return 0  #No detection

    _, x1, y1 = landmark_list[0]  # Thumb top
    _, x2, y2 = landmark_list[1]  # Index finger top
   
   # Line btween thimb and finger
    cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Highlighting 4 and 8
    cv2.circle(frame, (x1, y1), 6, (0, 255, 0), -1)
    cv2.circle(frame, (x2, y2), 6, (0, 255, 0), -1)

    # Calculate Euclidean distance between the two points
    distance = math.hypot(x2 - x1, y2 - y1)

    return distance

In [ ]:
# Setup Pycaw for volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# volume range
volMin, volMax = volume.GetVolumeRange()[:2]

In [ ]:
vid = cv2.VideoCapture(0)

while True:
    ret, frame = vid.read()
    frame = cv2.flip(frame, 1) 

    # Convert BGR to RGB as required by MediaPipe(cv2 bydefault is BGR)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    processed = hands.process(framergb) 

    # Get thumb & index tip landmarks for both hands
    left_landmark, right_landmark = get_landmark(frame, processed, draw, hands)

    # Brightness controller
    if len(left_landmark) == 2:
        left_distance = get_distance(frame, left_landmark) 
        brightness = np.interp(left_distance, [50, 200], [0, 100])  # Map to 0–100%
        set_brightness(brightness)  # Use screen_brightness_control to set screen brightness

    # Volume controller
    if len(right_landmark) == 2:
        right_distance = get_distance(frame, right_landmark)
        vol = int(min(max(right_distance, 20), 150)) # Clamp distance within 20–150 pixels
        volume_level = (vol - 20) / (150 - 20) * (volMax - volMin) + volMin # Map distance to system volume range
        volume.SetMasterVolumeLevel(volume_level, None)  # Set system volume

    cv2.imshow("Gesture Control", frame)

    if cv2.waitKey(1) & 0xFF == 27: # 27 is ord(esc)
        break

vid.release()
cv2.destroyAllWindows()

# optimize(avoid error on exit)
import comtypes
volume = None
devices = None
interface = None
comtypes.CoUninitialize()
